In [ ]:
!python -m pip install gurobipy

     |████████████████████████████████| 11.5 MB 7.8 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from gurobipy import *

**SCENARIO 1**

**Reading Drone Data**

In [ ]:
sets = pd.read_csv("/data/drone_info.csv")

**Reading Demands**

In [ ]:
Demands = pd.read_csv("/data/demand_info.csv")

#Demands = Demands[10:20].reset_index()

#Demands1 = Demands.where(Demands.Type < 4).dropna().reset_index()[["Type","Weight","Volume","Start","End","Origin","Dest","X","Y","Z"]].convert_dtypes(convert_integer = True)
#Demands2 = Demands.where(Demands.Type > 5).dropna().reset_index()[["Type","Weight","Volume","Start","End","Origin","Dest","X","Y","Z"]].convert_dtypes(convert_integer = True)

**Forming Nodes**

In [ ]:
locations = pd.read_csv("/data/locations.csv")

num = len(locations)                                  #Number of charging stations and warehouses
Nodes = locations[["X","Y","Z","CR"]]
Nodes = pd.concat([Nodes,Demands[["X","Y","Z"]]]).fillna(0).reset_index()[["X","Y","Z","CR"]]
Nodes["ID"] = Nodes.index

Demands["Dest"] = Demands.index + num

Demands.head()

,DID,Type,Weight,Volume,Start,End,Origin,X,Y,Z,Dest
0,1,1,1,200,600,3600,0,2009,1200,57,1
1,2,1,1,200,3600,10800,0,753,-1866,111,2
2,3,1,1,200,900,1500,0,568,1394,105,3
3,4,1,1,200,11700,14400,0,274,-268,73,4
4,5,1,1,200,4200,4800,0,-11,1508,40,5


In [ ]:
Nodes.to_csv("nodes.csv")

**Generating Routes**

In [ ]:
#Getting Routes
R = [[] for i in range(len(Demands))]

def feasible(i,j,k):
    if Nodes.X[k]>min(Nodes.X[i],Nodes.X[j]) and Nodes.X[k]<max(Nodes.X[i],Nodes.X[j]):
        pass
    else:
        return False
    
    if Nodes.Y[k]>(min(Nodes.Y[i],Nodes.Y[j])-100) and Nodes.Y[k]<(max(Nodes.Y[i],Nodes.Y[j])+100):
        pass
    else:
        return False
    
    if Nodes.Z[k]>(min(Nodes.Z[i],Nodes.Z[j])-100) and Nodes.Z[k]<(max(Nodes.Z[i],Nodes.Z[j])+100):
        pass
    else:
        return False
    
    return True


for i in range(len(Demands)):
     
    #Direct Route
    R[i].append([Demands.Origin[i],Demands.Dest[i]])
     
    #Intermediaries
    for j in range(len(Nodes)):
        if Demands.Dest[i]!=j and Demands.Origin[i]!=j:
            if feasible(Demands.Origin[i],Demands.Dest[i],j):
                R[i].append([Demands.Origin[i],j,Demands.Dest[i]])

Routes = R

**Generating Fxy Fzu Fzd**

In [ ]:


def xy(i,j):
    return ((Nodes.X[i] - Nodes.X[j])**2 + (Nodes.Y[i] - Nodes.Y[j])**2)**0.5

def zu(i,j):
    return max(0, Nodes.Z[j]-Nodes.Z[i])

def zd(i,j):
    return max(0, Nodes.Z[i]-Nodes.Z[j])

#Fxy table ready for nodes*nodes
Fxy = np.random.randint(0,2,size = (len(Nodes),len(Nodes)))

#Fzu table ready for nodes*nodes
Fzu = np.random.randint(0,2,size = (len(Nodes),len(Nodes)))

#Fzd table ready for nodes*nodes
Fzd = np.random.randint(0,2,size = (len(Nodes),len(Nodes)))

for i in np.arange(len(Nodes)):
    for j in np.arange(i,len(Nodes)):
        Fxy[i,j],Fzd[i,j],Fzu[i,j] = xy(i,j),zd(i,j),zu(i,j)
        #Fxy[i,j],Fzd[i,j],Fzu[i,j] = F(i,j)
        #Fxy[j,i],Fzd[j,i],Fzu[j,i] = Fxy[i,j],Fzd[i,j],Fzu[i,j]
    

In [ ]:
Fxy

array([[   0, 2340, 2012, 1505,  383, 1508, 1894, 2180, 2639,  974, 1270,
        2046,  608, 1815,  893, 2796, 1728, 2455, 2379, 1123, 2440, 1384,
        1763, 2275,  312, 1435, 2161, 2330, 2293, 2284, 1174],
       [   0,    0, 3313, 1454, 2272, 2043, 4092, 4520, 3807, 3291, 3451,
        4020, 2000,  883, 2799, 4193, 1751, 1975, 1329, 1306, 1021, 1382,
        2983, 1970, 2560, 2145, 4424, 3213, 4628, 3208, 1171],
       [   0,    1,    0, 3265, 1668, 3459, 3221, 2752, 4592, 1851, 2824,
        3653, 2550, 2439, 1159,  891, 1622, 4290, 2416, 2698, 3926, 3102,
        3758, 1772, 2222, 1167, 2087,  551, 2959, 4289, 2545],
       [   1,    1,    1,    0, 1687,  590, 2844, 3493, 2360, 2435, 2239,
        2659,  921, 1603, 2317, 4132, 2193, 1026, 2332,  592, 1022,  170,
        1532, 2673, 1564, 2270, 3633, 3407, 3552, 1773,  824],
       [   1,    1,    1,    0,    0, 1798, 2181, 2313, 3021, 1047, 1585,
        2391,  885, 1628,  629, 2487, 1397, 2687, 2112, 1200, 2533, 1542,
        

**Assumptions**

In [ ]:
max_trips = 5
max_trip = range(1,max_trips+1)
max_locales = range(1,max_trips + 2)

max_speed = 17
Time = 14400
charging_cost = 1.8

**LISTS**

In [ ]:
dr_chosen = []
demands_chosen = []
path = []
dr_schedules = []
charging_schedules = []

**CALLS**

In [ ]:
Drone_sets = sets.copy()
nodes = Nodes.copy()
demands = Demands.copy()
routes = Routes.copy()

count = 0
flag=1

**Algorithm 1**

In [ ]:
def Algo(): 
    mdl = Model("Algorithm 1")

    X = [(mdl.addVar(vtype = GRB.BINARY, name="X%s" % str([i])
                   .format(i))) for i in A ]

    Y = [(mdl.addVar(vtype = GRB.BINARY, name="Y%s" % str([i,j])
                   .format(i,j))) for i in A for j in J ]

    R = [(mdl.addVar(vtype = GRB.BINARY, name="R%s" % str([j,k])
                   .format(j,k))) for j in J for k in range(1,len(routes[j-1])+1)]

    Tarr = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tarr%s" % str([j,k,l])
                   .format(j,k,l), lb = 0.0, ub = Time)) for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))]

    Tdep = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tdep%s" % str([j,k,l])
                   .format(j,k,l), lb = 0.0, ub = Time)) for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))]

    Sxy = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Sxy%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    Szu = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Szu%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    Szd = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Szd%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    U = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="U%s" % str([i,m])
                   .format(i,m), lb = 0.0, ub = 1.0)) for i in A for m in max_trip]

    B = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="B%s" % str([i,q])
                   .format(i,q), lb = 0.0)) for i in A for q in max_locales]

    Ta = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Ta%s" % str([i,m])
                   .format(i,m), lb = 0.0, ub = Time)) for i in A for m in max_trip]

    Td = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Td%s" % str([i,m])
                   .format(i,m), lb = 0.0, ub = Time)) for i in A for m in max_trip]

    Z = [(mdl.addVar(vtype = GRB.BINARY, name="Z%s" % str([i,m])
                   .format(i,m))) for i in A for m in max_trip]

    #Recheck the correct number of legs you need for demand
    F = [(mdl.addVar(vtype = GRB.BINARY, name="F%s" % str([i,m,j,k,l])
                   .format(i,m,j,k,l))) for i in A for m in max_trip for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))]

    P = [(mdl.addVar(vtype = GRB.BINARY, name="P%s" % str([i,q,n])
                   .format(i,q,n))) for i in A for q in max_locales for n in N]

    C = [(mdl.addVar(vtype = GRB.BINARY, name="C%s" % str([i,m])
                   .format(i,m))) for i in A for m in max_trip]

    Tcu = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tcu%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    Tcd = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tcd%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    Tnet = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tnet%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    Txy = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Txy%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    Tzu = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tzu%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    Tzd = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tzd%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    E = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="E%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    Tflight = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tflight%s" % str([i])
                   .format(i), lb = 0.0)) for i in A ]

    Tchar = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tchar%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    TW1 = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="TW1%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    TW2 = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="TW2%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    H = [(mdl.addVar(vtype = GRB.INTEGER, name="H%s" % str([i,m])
                   .format(i,m), lb = 0.0)) for i in A for m in max_trip]

    Cvar = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Cvar%s" % str([i])
                   .format(i), lb = 0.0)) for i in A ]

    Cfix = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Cfix%s" % str([i])
                   .format(i), lb = 0.0)) for i in A ]

    Cenergy = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Cenergy", lb = 0.0))]

    mdl.update()

    mdl.modelSense = GRB.MAXIMIZE

    
    #########OBJECTIVES

    #Maximize Demand Satisfaction
    mdl.setObjectiveN(quicksum(mdl.getVarByName("Y" + str([i,j])) for i in A for j in J), 0, 1)

    #Minimize Costs
    mdl.setObjectiveN((-1 * quicksum(mdl.getVarByName("Cvar" + str([i])) for i in A) -
                       quicksum(mdl.getVarByName("Cfix" + str([i])) for i in A) -
                       mdl.getVarByName("Cenergy")), 1, 0)

    #Best Path
    #mdl.setObjectiveN(quicksum(mdl.getVarByName("Z" + str([i,m])) for i in A for m in max_trip), 2, 0)

    

    #############CONSTRAINTS
    
    
    #DIRECT BOUNDS
    mdl.addConstrs((mdl.getVarByName("Tdep"+str([j,k,l])) <= 
                    Time * mdl.getVarByName("R"+str([j,k]))
                    for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))), name = "Max_bound_to_Tdep");

    mdl.addConstrs((mdl.getVarByName("Tarr"+str([j,k,l])) <= 
                    Time * mdl.getVarByName("R"+str([j,k])) 
                    for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))), name = "Max_bound_to_Tarr");
    
    mdl.addConstrs((mdl.getVarByName("Tarr"+str([j,k,l])) <= 
                    demands.End[j-1] * mdl.getVarByName("R"+str([j,k])) 
                    for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))), name = "Tarr_before_End_Time");

    mdl.addConstrs((mdl.getVarByName("Tdep"+str([j,k,l])) >= 
                    demands.Start[j-1] * mdl.getVarByName("R"+str([j,k]))
                    for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))), name = "Tdep_after_Start_Time");

    mdl.addConstrs((mdl.getVarByName("U"+str([i,m])) == sum(mdl.getVarByName("F"+str([i,m,j,k,l])) * demands.Weight[j-1] for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1])))/drone_sets.Payload[i-1] 
                    for i in A for m in max_trip), name = "Fractional");

    mdl.addConstrs((mdl.getVarByName("Sxy"+str([i,m])) == (max_speed - mdl.getVarByName("U"+str([i,m])) * drone_sets.P[i-1]) 
                    for i in A for m in max_trip), name = "Sxy");

    mdl.addConstrs((mdl.getVarByName("Szu"+str([i,m])) == (max_speed - mdl.getVarByName("U"+str([i,m])) * drone_sets.Q[i-1]) 
                    for i in A for m in max_trip), name = "Szu");

    mdl.addConstrs((mdl.getVarByName("Szd"+str([i,m])) == (max_speed + mdl.getVarByName("U"+str([i,m])) * drone_sets.Q[i-1]) 
                    for i in A for m in max_trip), name = "Szd");
    
    mdl.addConstrs((mdl.getVarByName("Td"+str([i,m])) <= 
                    mdl.getVarByName("X"+str([i])) * Time for i in A for m in max_trip), name = "Setting_bound_on_Td_i");

    mdl.addConstrs((mdl.getVarByName("Ta"+str([i,m])) <= 
                    mdl.getVarByName("X"+str([i])) * Time for i in A for m in max_trip), name = "Setting_bound_on_Ta_i");
    
    mdl.addConstrs((
                    (mdl.getVarByName("Txy"+str([i,m])) * mdl.getVarByName("Sxy"+str([i,m]))) == 
                    sum(Fxy[a][b] * 
                    mdl.getVarByName("P"+str([i,m,a])) * 
                    mdl.getVarByName("P"+str([i,m+1,b])) for a in N for b in N) 
                    for i in A for m in max_trip ), name = "Txy");

    mdl.addConstrs((
                    (mdl.getVarByName("Tzu"+str([i,m])) * mdl.getVarByName("Szu"+str([i,m]))) == 
                    sum(Fzu[a][b] * 
                    mdl.getVarByName("P"+str([i,m,a])) * 
                    mdl.getVarByName("P"+str([i,m+1,b])) for a in N for b in N) 
                    for i in A for m in max_trip ), name = "Tzu");

    mdl.addConstrs((
                    (mdl.getVarByName("Tzd"+str([i,m])) * mdl.getVarByName("Szd"+str([i,m]))) == 
                    sum(Fzd[a][b] * 
                    mdl.getVarByName("P"+str([i,m,a])) * 
                    mdl.getVarByName("P"+str([i,m+1,b])) for a in N for b in N) 
                    for i in A for m in max_trip ), name = "Tzd");

    mdl.addConstrs((mdl.getVarByName("Tnet"+str([i,m])) == 
                    (mdl.getVarByName("Txy"+str([i,m]))+mdl.getVarByName("Tzu"+str([i,m]))+mdl.getVarByName("Tzd"+str([i,m])))
                    for i in A for m in max_trip), name = "Tnet");
    
    mdl.addConstrs((mdl.getVarByName("Ta"+str([i,m])) == 
                    (mdl.getVarByName("Td"+str([i,m]))+mdl.getVarByName("Tnet"+str([i,m])))
                    for i in A for m in max_trip), name = "Ta");

    mdl.addConstrs((mdl.getVarByName("Td"+str([i,m+1])) >= 
                    mdl.getVarByName("Ta"+str([i,m]))
                    for i in A for m in max_trip[:-1]), name = "Next_dep_after_last_arrival");

    mdl.addConstrs((mdl.getVarByName("Tflight"+str([i])) == 
                    sum(mdl.getVarByName("Tnet"+str([i,m])) for m in max_trip)
                    for i in A ), name = "Tflight");

    mdl.addConstrs((mdl.getVarByName("Tcu"+str([i,m])) <= 
                    Time*mdl.getVarByName("C"+str([i,m])) for i in A for m in max_trip), 
                    name = "Bound_on_Charge_Start");

    mdl.addConstrs((mdl.getVarByName("Tcd"+str([i,m])) <= 
                    Time*mdl.getVarByName("C"+str([i,m])) for i in A for m in max_trip), 
                    name = "Bound_on_Charge_End");

    mdl.addConstrs((mdl.getVarByName("Tcu"+str([i,m])) >= 
                    mdl.getVarByName("C"+str([i,m]))*mdl.getVarByName("Ta"+str([i,m]))
                    for i in A for m in max_trip),
                    name = "Bound_on_Charge_Start2");

    mdl.addConstrs((mdl.getVarByName("Tcd"+str([i,m])) <= 
                    mdl.getVarByName("C"+str([i,m]))*mdl.getVarByName("Td"+str([i,m+1]))
                    for i in A for m in max_trip[:-1]),
                    name = "Bound_on_Charge_End2");

    mdl.addConstrs((mdl.getVarByName("C"+str([i,max_trips])) == 0 for i in A), name = "No_charging_in_last_node");

    mdl.addConstrs((
                (mdl.getVarByName("F"+str([i,m,j,k,l])) == 1) >>
                (mdl.getVarByName("Ta"+str([i,m])) == mdl.getVarByName("Tarr"+str([j,k,l])))
                for i in A 
                for j in J 
                for m in max_trip
                for k in range(1,len(routes[j-1])+1) 
                for l in range(1,len(routes[j-1][k-1]))), 
                name = "Connecting_arrival_times");

    mdl.addConstrs((
                (mdl.getVarByName("F"+str([i,m,j,k,l])) == 1) >>
                (mdl.getVarByName("Td"+str([i,m])) == mdl.getVarByName("Tdep"+str([j,k,l])))
                for i in A 
                for j in J 
                for m in max_trip
                for k in range(1,len(routes[j-1])+1) 
                for l in range(1,len(routes[j-1][k-1]))), 
                name = "Connecting_departure_times");
    
    mdl.addConstrs((
                (mdl.getVarByName("F"+str([i,m,j,k,l])) == 1) >>
                (mdl.getVarByName("P"+str([i,m,a])) == 1 )
                for i in A 
                for j in J 
                for a in N
                for m in max_trip
                for k in range(1,len(routes[j-1])+1) 
                for l in range(1,len(routes[j-1][k-1]))
                if a == routes[j-1][k-1][l-1]
                ), 
                name = "Connecting_departure_nodes");

    mdl.addConstrs((
                (mdl.getVarByName("F"+str([i,m,j,k,l])) == 1) >>
                (mdl.getVarByName("P"+str([i,m+1,b])) == 1)
                for i in A 
                for j in J 
                for b in N
                for m in max_trip
                for k in range(1,len(routes[j-1])+1) 
                for l in range(1,len(routes[j-1][k-1]))
                if b == routes[j-1][k-1][l]), 
                name = "Connecting_arrival_nodes");

    mdl.addConstrs((mdl.getVarByName("P"+str([i,1,drone_sets.Wh[i-1]-1])) == mdl.getVarByName("X"+str([i])) for i in A), name = "Initial_warehouse");

    mdl.addConstrs((mdl.getVarByName("P"+str([i,q,n])) <= mdl.getVarByName("X"+str([i])) for i in A for q in max_locales for n in N), name = "Not_selected_no_path");

    mdl.addConstrs((sum(mdl.getVarByName("P"+str([i,q,n])) for n in N) == mdl.getVarByName("X"+str([i])) for i in A for q in max_locales), name = "One_node_if_selected");

    #ENERGY CONSTRAINTS AND CALCULATIONS
    mdl.addConstrs((mdl.getVarByName("TW1"+str([i,m])) ==
                    (mdl.getVarByName("X"+str([i])) *drone_sets.Weight[i-1] + 
                     sum(mdl.getVarByName("F"+str([i,m,j,k,l])) * demands.Weight[j-1] for j in J 
                    for k in range(1,len(routes[j-1])+1) 
                    for l in range(1,len(routes[j-1][k-1]))))
                    for i in A for m in max_trip), name = "TW1");


    mdl.addConstrs((
                mdl.getVarByName("TW2"+str([i,m])) == 
                (drone_sets.A[i-1] * mdl.getVarByName("Tnet"+str([i,m])) + 
                 drone_sets.B[i-1] * (mdl.getVarByName("Sxy"+str([i,m]))*mdl.getVarByName("Txy"+str([i,m])) + mdl.getVarByName("Szu"+str([i,m]))*mdl.getVarByName("Tzu"+str([i,m])) + mdl.getVarByName("Szd"+str([i,m]))*mdl.getVarByName("Tzd"+str([i,m]))) +
                 drone_sets.C[i-1] * sum(Fzu[a][b] * mdl.getVarByName("P"+str([i,m,a])) * mdl.getVarByName("P"+str([i,m+1,b])) for a in N for b in N) 
                ) for i in A for m in max_trip ), name = "TW2");

    mdl.addConstrs((
                mdl.getVarByName("E"+str([i,m])) == mdl.getVarByName("TW1"+str([i,m]))*mdl.getVarByName("TW2"+str([i,m]))
                for i in A for m in max_trip), name = "Energy_consumption");
    
    mdl.addConstrs((mdl.getVarByName("B"+str([i,1])) == drone_sets.Battery[i-1] for i in A), name = "Start_with_full_battery");

    mdl.addConstrs((mdl.getVarByName("B"+str([i,q])) <= drone_sets.Battery[i-1] for i in A for q in max_locales), name = "Max_battery_bound");
    
    mdl.addConstrs((
            mdl.getVarByName("B"+str([i,m+1])) == 
            (mdl.getVarByName("B"+str([i,m]))
             -mdl.getVarByName("E"+str([i,m])) 
             +sum(mdl.getVarByName("Tchar"+str([i,m])) * mdl.getVarByName("P"+str([i,m+1,a])) * nodes.CR[a] for a in N)) 
             for i in A for m in max_trip ), name = "Update battery");
    
    mdl.addConstrs((
                sum(mdl.getVarByName("F"+str([i,m,j,k,l])) * demands.Weight[j-1] for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))) <=
                drone_sets.Payload[i-1] for i in A for m in max_trip), name = "Weight");

    mdl.addConstrs((
                sum(mdl.getVarByName("F"+str([i,m,j,k,l])) * demands.Volume[j-1] for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))) <=
                drone_sets.Volume[i-1] for i in A for m in max_trip), name = "Volume");

    mdl.addConstrs((
                sum(mdl.getVarByName("F"+str([i,m,j,k,l])) for j in J for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))) <=
                drone_sets.Slots[i-1] for i in A for m in max_trip), name = "Carrying_Slots");    
    
    
    mdl.addConstr((sum(mdl.getVarByName("X"+str([i])) 
                        for i in A) <= 1), name = "One_Drone");
    
    mdl.addConstrs((mdl.getVarByName("Y"+str([i,j])) 
                    <= mdl.getVarByName("X"+str([i])) for i in A for j in J), name = "No_Drone_No_Demand");
    
    mdl.addConstrs((sum(mdl.getVarByName("Y"+str([i,j])) for j in J) 
                    >= mdl.getVarByName("X"+str([i])) for i in A), name = "Some_Demand");
    
    mdl.addConstrs((sum(mdl.getVarByName("R"+str([j,k])) for k in range(1,len(routes[j-1])+1))
                    == sum(mdl.getVarByName("Y"+str([i,j])) for i in A) for j in J ), name = "Should_Route_Be_Taken");
    
    
    
    #Costs
    mdl.addConstrs((mdl.getVarByName("Cvar"+str([i])) == 
                    drone_sets.Mvar[i-1]*mdl.getVarByName("Tflight"+str([i])) for i in A),
                    name = "Variable");

    mdl.addConstrs((mdl.getVarByName("Cfix"+str([i])) ==
                    drone_sets.Mfix[i-1]*mdl.getVarByName("X"+str([i])) for i in A),
                    name = "Fixed");

    mdl.addConstr((mdl.getVarByName("Cenergy") ==
                    charging_cost * 
                    quicksum(mdl.getVarByName("Tchar"+str([i,m]))*mdl.getVarByName("P"+str([i,m,n]))*nodes.CR[n] for n in N for m in max_trip for i in A)),
                    name = "EnergyCost");
    
    
    
    

    

    


    


    

    
    
    
    

    


   



    #BASIC
    mdl.addConstrs((mdl.getVarByName("F"+str([i,m,j,k,l])) <= mdl.getVarByName("R"+str([j,k])) 
                for i in A 
                for j in J 
                for m in max_trip
                for k in range(1,len(routes[j-1])+1) 
                for l in range(1,len(routes[j-1][k-1]))), 
                name = "Demand_route_should_have_been_taken");


    mdl.addConstrs((mdl.getVarByName("F"+str([i,m,j,k,l])) <= mdl.getVarByName("X"+str([i])) 
                for i in A 
                for j in J 
                for m in max_trip
                for k in range(1,len(routes[j-1])+1) 
                for l in range(1,len(routes[j-1][k-1]))), 
                name = "Drone_should_have_been_taken");

    

    


    #ADVANCED and ASSIGNMENT

    mdl.addConstrs((
                sum(mdl.getVarByName("F"+str([i,m,j,k,l])) for m in max_trip for k in range(1,len(routes[j-1])+1) for l in range(1,len(routes[j-1][k-1]))) == 
                sum(mdl.getVarByName("R"+str([j,p])) * (len(routes[j-1][p-1])-1) for p in range(1,len(routes[j-1]) + 1)) *
                    mdl.getVarByName("Y"+str([i,j])) 
                for i in A 
                for j in J), 
                name = "Selection_must");


    mdl.addConstrs((
                (mdl.getVarByName("F"+str([i,m,j,k,1])) == 1) >>
                (mdl.getVarByName("F"+str([i,m+l-1,j,k,l])) == 1)
                for i in A 
                for j in J 
                for k in range(1,len(routes[j-1])+1) 
                for l in range(1,len(routes[j-1][k-1]))
                for m in max_trip[:max_trips - len(routes[j-1][k-1][:-1]) + 1]
                ), name = "Creating_Connectivity");

    #BASIC

    





    #ADVANCED



    

    


    #BASIC

    mdl.addConstrs((mdl.getVarByName("C"+str([i,m])) <= 
                    mdl.getVarByName("X"+str([i])) for i in A for m in max_trip),
                    name = "Selected_drones_charge");

    #Correct for warehouses and charging stations during final code
    mdl.addConstrs((mdl.getVarByName("C"+str([i,m])) * (sum(mdl.getVarByName("P"+str([i,m+1,a])) for a in range(0,num)) - 1) >= 0
                    for i in A for m in max_trip ), name = "Charging_station_only_charge")

    
    mdl.addConstrs((mdl.getVarByName("Tchar"+str([i,m])) == 
                    mdl.getVarByName("Tcd"+str([i,m]))-mdl.getVarByName("Tcu"+str([i,m]))
                    for i in A for m in max_trip),
                    name = "Charging_Time");


    #Minimum charging time for added constraints
    mdl.addConstrs((
                    (mdl.getVarByName("C"+str([i,m])) == 1) >>
                    (mdl.getVarByName("Tchar"+str([i,m])) >= 100)
                    for i in A for m in max_trip),
                    name = "Charging_Time_Minimum");
    
    #BASIC
    




                 





    


    #Constraint 15

    '''mdl.addConstrs((mdl.getVarByName("H"+str([i,m])) == 
                    sum(mdl.getVarByName("F"+str([i,u,j,k,l]))
                        for j in J
                        for k in range(1,len(routes[j-1])+1) 
                        for l in range(1,len(routes[j-1][k-1])) 
                        for u in range(m,max_trips+1)
                       ) 
                    for i in A for m in max_trip), name = "H_constr"); 

    mdl.addConstrs((
                   (-1000 * mdl.getVarByName("H"+str([i,m])) + mdl.getVarByName("X"+str([i]))) <=
                   mdl.getVarByName("Z"+str([i,m]))
                   for i in A for m in max_trip), name = "Z_set1");

    mdl.addConstrs((mdl.getVarByName("X"+str([i])) >=
                   mdl.getVarByName("Z"+str([i,m]))
                   for i in A for m in max_trip), name = "Z_set2");'''
    
    
    
    
    
    #mdl.addConstrs((
    #                (mdl.getVarByName("P"+str([i,m+1,a])) == 1) >>
    #                (mdl.getVarByName("Tcu"+str([i,m])) - charging_schedules[b][2])*(mdl.getVarByName("Tcd"+str([i,m])) - charging_schedules[b][2]) >= 0.1
    #                for b in range(len(charging_schedules)) for i in A for m in max_trip for a in N if charging_schedules[b][0] == a), name = "Constr1");
    
    #mdl.addConstrs((
    #                (mdl.getVarByName("P"+str([i,m+1,a])) == 1) >>
    #                (mdl.getVarByName("Tcu"+str([i,m])) - charging_schedules[b][1])*(mdl.getVarByName("Tcd"+str([i,m])) - charging_schedules[b][1]) >= 0.1
    #                for b in range(len(charging_schedules)) for i in A for m in max_trip for a in N if charging_schedules[b][0] == a), name = "Constr2");
    

    mdl.params.TimeLimit = 200
    mdl.params.NonConvex = 2
    mdl.update()
    mdl.optimize()
    
    return mdl     

**WHILE LOOP CALL**

In [ ]:
flag = 1

while(len(Drone_sets)>0 and len(demands)>0 and flag==1):
    count = count + 1
    print("\n\n\nIteration Number :",count,"\n\n\n")
    
    #print("Demands\n\n\n")
    #print(demands)
    #print("\n\n\nDrone\n\n\n")
    #print(Drone_sets)
    
    #SETS
    
    drone_sets = Drone_sets[:int(count/2)+1].copy()
    #drone_sets = Drone_sets.copy()
    #Number of drone sets possible
    A = range(1,len(drone_sets)+1)
    
    #Number of nodes
    N = range(len(nodes))
    
    #Number of unfulfilled demands
    J = range(1,len(demands)+1)
    
    
    
    
    mdl = Algo()
    
    if mdl.status == GRB.INFEASIBLE:
        print("Infeasible")
        flag==0
        mdl.computeIIS()
        mdl.write("m.ilp")
        
    for i in A:
        c = mdl.getVarByName("X"+str([i]))
        if(c.x)==1:
            for m in max_trip:
                #print(mdl.getVarByName("E"+str([i,m])))
                #print(mdl.getVarByName("Txy"+str([i,m])))
                #print(mdl.getVarByName("Tzu"+str([i,m])))
                #print(mdl.getVarByName("Tzd"+str([i,m])))
                for a in N:
                    for b in N:
                        p1 = mdl.getVarByName("P"+str([i,m,a]))
                        p2 = mdl.getVarByName("P"+str([i,m+1,b]))
                        if p1.x==1 and p2.x==1:
                            #print(p1,p2)
                            pass
                
    #Drone Chosen
    f=0
    for i in A:
        c = mdl.getVarByName("X"+str([i]))
        if(c.x)==1:
            dr_chosen.append(["T"+str(drone_sets.Type[i-1]),"WH"+str(drone_sets.Wh[i-1])])
            Drone_sets.Count[i-1] = Drone_sets.Count[i-1] - 1
            f=1
    if f==0 and count > 30:
        print("\n\n\n*********************************No drone selected!!!!")
        flag = 0        
                
    #Path Taken
    for i in A:
        c = mdl.getVarByName("X"+str([i]))
        if(c.x)==1:
            list2 = ["T"+str(drone_sets.Type[i-1]),"WH"+str(drone_sets.Wh[i-1])]
            for q in max_locales:
                for n in N:
                    t = mdl.getVarByName("P"+str([i,q,n]))
                    if (t.x == 1):
                        list2.append(Nodes.ID[n])
                        
            path.append(list2 )
            
            
    #Drone Schedules
    for i in A:
        c = mdl.getVarByName("X"+str([i]))
        if(c.x)==1:
            list3 = [drone_sets.Type[i-1],drone_sets.Wh[i-1]]
            for m in max_trip:
                t1 = mdl.getVarByName("Ta"+str([i,m]))
                t2 = mdl.getVarByName("Td"+str([i,m]))
                list3.append([t2.x,t1.x])
            
            dr_schedules.append(list3)
            
    
    #Demands Fulfilled
    demandsFulfilled = []
    for i in A:
        for j in J:
            c = mdl.getVarByName("Y"+str([i,j]))
            if (c.x == 1):
                demands_chosen.append(demands.Dest[j-1])
                demandsFulfilled.append(j-1)
    cnt = 0            
    for x in demandsFulfilled:
        demands.drop(x, inplace = True, axis = 0)
        del routes[x-cnt]
        cnt = cnt+1
    demands = demands[["DID", "Type", "Weight", "Volume", "Start", "End", "Origin", "X","Y","Z","Dest"]]
    demands.reset_index(inplace = True)
    
    
    #Charging Schedules
    for i in A:
        c = mdl.getVarByName("X"+str([i]))
        if(c.x)==1:
            for m in max_trip:
                t = mdl.getVarByName("C"+str([i,m]))
                if (t.x == 1):
                    node = -1
                    for a in N:
                        p = mdl.getVarByName("P"+str([i,m+1,a]))
                        if p.x == 1:
                            node = a
                    
                    s1 = mdl.getVarByName("Tcu"+str([i,m]))
                    s2 = mdl.getVarByName("Tcd"+str([i,m]))
            
                    charging_schedules.append([node,s1.x,s2.x])

            
    #Drone Chosen
    for i in A:
        c = mdl.getVarByName("X"+str([i]))
        if(c.x)==1:
            if Drone_sets.Count[i-1] == 0:
                Drone_sets.drop(i-1, inplace = True)
                Drone_sets = Drone_sets[["Type", "Wh", "Count", "Battery", "Weight", "Payload", 
                                     "Volume", "Slots", "Mfix", "Mvar", "P", 
                                     "Q", "A", "B", "C"]]
                Drone_sets.reset_index(inplace = True)                
                
    print("\nAfter",count,"th iteration\n")
    print("Drone",dr_chosen)
    print("Demands", demands_chosen)
    #print("Paths", path)
    #print("Charging Schedules",charging_schedules)
    #print("Drone Schedule", dr_schedules)
                        
    




Iteration Number : 2 



Set parameter TimeLimit to value 200
Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads


GurobiError: ignored

In [ ]:
demands_chosen

[4,
 5,
 24,
 25,
 3,
 6,
 30,
 2,
 16,
 27,
 1,
 7,
 26,
 9,
 12,
 14,
 10,
 11,
 13,
 15,
 17,
 29,
 19,
 21,
 22,
 20,
 23,
 28,
 8,
 18]

In [ ]:
path

[['T1', 'WH1', 0, 5, 0, 4, 0],
 ['T2', 'WH1', 0, 25, 0, 24, 0],
 ['T2', 'WH1', 0, 3, 0, 30, 0, 6],
 ['T3', 'WH1', 0, 16, 0, 2, 27, 27],
 ['T3', 'WH1', 0, 1, 0, 0, 7],
 ['T4', 'WH1', 0, 12, 0, 14, 0, 9],
 ['T4', 'WH1', 0, 13, 0, 10, 0, 11],
 ['T4', 'WH1', 0, 17, 0, 15, 0, 29],
 ['T5', 'WH1', 0, 19, 0, 21, 0, 22],
 ['T6', 'WH1', 0, 23, 0, 28, 0],
 ['T6', 'WH1', 0, 0, 8, 0, 18, 18]]

In [ ]:
dr_chosen

[['T1', 'WH1'],
 ['T2', 'WH1'],
 ['T2', 'WH1'],
 ['T3', 'WH1'],
 ['T3', 'WH1'],
 ['T4', 'WH1'],
 ['T4', 'WH1'],
 ['T4', 'WH1'],
 ['T5', 'WH1'],
 ['T6', 'WH1'],
 ['T6', 'WH1']]

In [ ]:
dr_schedules

[[1,
  1,
  [4199.999999999984, 4292.142857142841],
  [4292.142857142841, 4383.201680672253],
  [11699.99999999996, 11727.142857142817],
  [11727.142857142817, 11753.966386554583],
  [13199.999999999918, 13319.345238095157]],
 [2,
  1,
  [1800.0, 1899.2291666666665],
  [1899.2291666666665, 1986.9938725490194],
  [5374.075, 5400.0],
  [14290.681862745098, 14313.858333333334],
  [14313.858333333334, 14400.0]],
 [2,
  1,
  [899.9999999999164, 996.5376237622995],
  [996.537623762264, 1091.2435061152053],
  [9599.999999999942, 9686.141666666608],
  [14203.295352358766, 14279.76594059406],
  [14279.76594059406, 14400.0]],
 [3,
  1,
  [3599.9999999999864, 3713.702702702689],
  [8893.941176470576, 8999.999999999989],
  [8999.999999999989, 9141.533333333322],
  [9141.533333333324, 9178.36510903426],
  [9178.365109034221, 9178.365109034221]],
 [3,
  1,
  [3169.7018931224543, 3313.1121495327106],
  [3313.1121495327106, 3454.1121495327106],
  [3454.1121495327106, 3600.0000000000005],
  [3600.00000

In [ ]:
charging_schedules

[[0, 3315.862903225807, 3415.862903225803]]

In [ ]:
Demands.drop(np.array(demands_chosen)-1,axis = 0)

,DID,Type,Weight,Volume,Start,End,Origin,X,Y,Z,Dest


In [ ]:
drone_sets

,Type,Wh,Count,Battery,Weight,Payload,Volume,Slots,Mfix,Mvar,P,Q,A,B,C
0,1,1,1,2000,2.0,5,200,1,10,5,1,1,0.01,0.02,0.01
1,2,1,3,2500,2.5,6,500,1,10,5,2,1,0.02,0.03,0.01
2,3,1,3,3000,3.0,7,1000,2,10,5,2,2,0.08,0.03,0.01
3,4,1,2,4000,3.5,8,2000,2,10,5,3,2,0.05,0.05,0.02
4,5,1,4,5000,4.0,9,3000,2,10,5,5,3,0.06,0.04,0.03
5,6,1,1,10000,5.0,10,5000,4,10,5,4,4,0.04,0.03,0.03


In [ ]:
drone_sets[:7]

,Type,Wh,Count,Battery,Weight,Payload,Volume,Slots,Mfix,Mvar,P,Q,A,B,C
0,1,1,1,2000,2.0,5,200,1,10,5,1,1,0.01,0.02,0.01
1,2,1,3,2500,2.5,6,500,1,10,5,2,1,0.02,0.03,0.01
2,3,1,3,3000,3.0,7,1000,2,10,5,2,2,0.08,0.03,0.01
3,4,1,2,4000,3.5,8,2000,2,10,5,3,2,0.05,0.05,0.02
4,5,1,4,5000,4.0,9,3000,2,10,5,5,3,0.06,0.04,0.03
5,6,1,1,10000,5.0,10,5000,4,10,5,4,4,0.04,0.03,0.03


In [ ]:
Demands

,DID,Type,Weight,Volume,Start,End,Origin,X,Y,Z,Dest
0,1,1,1,200,600,3600,0,2009,1200,57,1
1,2,1,1,200,3600,10800,0,753,-1866,111,2
2,3,1,1,200,900,1500,0,568,1394,105,3
3,4,1,1,200,11700,14400,0,274,-268,73,4
4,5,1,1,200,4200,4800,0,-11,1508,40,5
5,6,1,1,200,13200,14400,0,-1894,-30,111,6
6,7,1,1,200,13800,14400,0,-1888,-1091,114,7
7,8,4,2,1200,2700,3600,0,-1711,2010,139,8
8,9,4,2,1200,12600,14400,0,-682,-696,76,9
9,10,4,2,1200,5400,6300,0,-1266,109,117,10
